In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator

import src.data_cleaning as dc

In [2]:
df = dc.create_df('bigml_59c28831336c6604c800002a.csv')

In [6]:
class Dataprep(BaseEstimator):
    
    def __init__(self):
        pass
        
    def fit(self, X, y=None):
        return self 
    
    def transform(self, X, y=None):
        
        df = X.copy()
        df['international plan'] = df['international plan'] == 'yes'
        df['voice mail plan'] = df['voice mail'] == 'yes'
    
        df['account months'] = np.ceil(df['account length']/30).astype(int)
        df['total minutes'] = df['total day minutes']+df['total eve minutes']+df['total night minutes']+df['total intl minutes']
        df['total calls'] = df['total day calls']+df['total eve calls']+df['total night calls']+df['total intl calls']
        df['total cost'] = df['total day charge']+df['total eve charge']+df['total night charge']+df['total intl charge']
        df['cost per day'] = df['total cost']/df['account length']
    
        df.drop(['state', 'phone number', 'area code'], axis=1, inplace=True)
    
        return df

In [8]:
dataprep = Dataprep()
rfc = RandomForestClassifier(random_state=42, n_estimators=100)

pipe = Pipeline(steps=[
    ("dataprep", Dataprep())
    ("model", rfc)
])

TypeError: 'tuple' object is not callable

In [ ]:
pipe.fit(X_train, y_train)

In [ ]:
pipe.predict(X_test)

In [ ]:
pipe.transform(X_train)